# 预处理数据 (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

训练代码样例，做了一次forward和backward

In [ ]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

## 数据集加载

Hugging Face的datasets Hub上（https://huggingface.co/datasets）
有很多数据集，也可以上传数据集。

官网推荐读者尝试使用datasets库来加载和处理数据集（问题是国内加载非常的慢，可以用kaggle的kernel，速度哇塞！）

此处使用MRPC数据集，该数据集是英文NLP基准评测GLUE benchmark（https://gluebenchmark.com/leaderboard）
中的一个数据集，该评测包含10个文本分类任务，另外还有升级版的super GLUE，中文有CLUE，大家可以自行百度。

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc") 
raw_datasets

d:\bag\Anaconda3.2011.11\envs\transformer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset glue downloaded and prepared to C:/Users/pb078553/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 198.92it/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

数据集默认下载到C:\Users\xxx\.cache\huggingface\datasets\glue\mrpc。

数据集包括训练、验证和测试数据集（训练数据用来训练模型、验证数据用来选择模型，测试数据用来测试模型的泛化能力）。每个数据集都包含features和num_rows两个key，其中num_rows记录数据量，features存储实际数据，该数据中有两个文本（sentence1和sentence2），标签label和数据的序号（idx唯一的给数据一个序号）。由上可知，我们有3,668个训练数据、408个验证数据和1,725个测试数据。

In [10]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0] # 0是行号

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [11]:
raw_train_dataset[1]

{'sentence1': "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
 'sentence2': "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .",
 'label': 0,
 'idx': 1}

In [12]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

从上述代码运行结果可知，label是ClassLabel类型，其对应的标签名称存储在names对象中，其中数值0表示not_equivalent，1表示equivalent。

In [13]:
from transformers import AutoTokenizer

# 英文版的BERT在预训练支持同时输入两个文本
# checkpoint = "bert-base-uncased"
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint) 
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

d:\bag\Anaconda3.2011.11\envs\transformer\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pb078553\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

 当输入两个文本时，tokenizer会将两个文本编码为[CLS] 文本1 [SEP] 文本2 [SEP]的格式（如果是一个句子，那么会编码为[CLS] 文本1 [SEP]）。
 
 token_type_ids的作用就是完成编码后的第一个句子和第二个句子的对齐。不是所有的模型的输入都需要token_type_ids作为输入,例如DistilBERT。此处使用的bert-base-uncased模型必须要token_type_ids。

 tokenizer会自动构建token_type_ids。



## 数据集批处理

 下面的代码是一次性对所有的训练数据进行分析。一般针对比较小的数据集，比较大数据集合要使用dataset的map函数来操作，防止内存不够。

In [16]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

使用Dataset.map()函数，首先需要定义一个lambda或如下函数。输入为example，是一个字典，并且返回input_ids、attention_mask和token_type_ids的新字典。

如果example中的value是一个列表（因为有时会一次性输入多个数据，即批量操作），那么函数也会准确执行（必须使用Dataset对象）。由于tokenizer使用Rust编写和优化了分词器，因此批量处理的速度非常快，当然也和你的内存和批处理数据的多少有关系，总之很快。

下方代码中的tokenizer函数没有设置padding参数，这是因为一次对所有的数据进行padding是低效的。没有必要按照最长的那个数据对所有数据进行padding，而是在一个批次中进行padding，这样可以减少padding的长度，也有利于模型的计算（对于文本生成，padding越长，生成的效果越好）

In [17]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], 
                     example["sentence2"], 
                     truncation=True
                     )

设置batched=True会一次性处理一批数据（默认batch_size=1000）。一般批尺寸越大，运算越快（具体和计算机配置有关系）。

设置num_proc来配置多线程处理。默认配置就已经很快了。如果在你自己的数据集上处理的很慢，可以考虑修改num_proc参数来提高数据处理速度。

In [18]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1725
    })
})

执行完成上述操作以后，会给dataset添加数据。增加了input_ids、attention_mask和token_type_ids等field（这些数据对应各行文本对的分词结果）。

## 动态补齐Dynamic padding

整个数据集补齐采用整个数据集中最长的文本来补齐，按批次补齐采用各个批次中最长文本来补齐，显然后者补齐的数据长度要小一些。

DataCollatorWithPadding进行批量补齐操作（同时作用于input_ids、attention_mask和token_type_ids）。该函数需要一个tokenizer作为参数导入（通过该参数函数可以得到padding_token_id，以及是从左边开始补齐还是右边开始补齐，或者其他的模型输入要求）。

In [19]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

 为了测试补齐函数的功能，我们先构造一个批数据，然后移除掉（idx、sentence1、sentence2，并且没有转换为tensors的字符串），然后观察下input_ids的长度

In [20]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

我们得到了不同input_ids长度的批数据，最长67，最短32。动态补齐会将所有数据补齐到67。

In [21]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}